#Introducción a SpaCy

###Ejercicio 1: Objeto Token

In [1]:
# Importa la clase de lenguaje "Spanish"
from spacy.lang.es import Spanish

# Crea el objeto nlp
nlp = Spanish()

# Procesa un texto
doc = nlp("¿Cómo estás?")

# Imprime en pantalla el texto del documento
print(doc.text)

¿Cómo estás?


###Ejercicio 2: Objeto Token y Objeto Span

In [ ]:
# Procesa el texto
doc = nlp("Me gustan las panteras negras y los leones.")

# Selecciona el primer token
first_token = doc[0]

# Imprime en pantalla el texto del token
print(first_token.text)

Me


In [ ]:
# Un slice del Doc para "panteras negras"
panteras_negras = doc[3:5]
print(panteras_negras.text)

# Un slice del Doc para "panteras negras y los leones" (sin el ".")
panteras_negras_y_leones = doc[3:8]
print(panteras_negras_y_leones.text)

panteras negras
panteras negras y los leones


###Ejercicio 3: Encontrar símbolos en el texto tokenizado

In [ ]:
# Procesa el texto
doc = nlp(
    "En 1990, más del 60 % de las personas en Asia del Este se encontraban "
    "en extrema pobreza. Ahora, menos del 4 % lo están."
)

# Itera sobre los tokens en el doc
for token in doc:
    # Revisa si el token parece un número
    if token.like_num:
        # Obtén el próximo token en el documento
        next_token = doc[token.i + 1]
        # Revisa si el texto del siguiente token es igual a '%'
        if next_token.text == "%":
            print("Porcentaje encontrado:", token.text)

Porcentaje encontrado: 60
Porcentaje encontrado: 4


###Ejercicio 4: Cargar modelos preentrenados

Se pueden encontrar aquí las instrucciones de cómo importarlos https://spacy.io/models

In [2]:
!pip3 install https://github.com/explosion/spacy-models/releases/download/es_core_news_sm-2.2.0/es_core_news_sm-2.2.0.tar.gz

     |████████████████████████████████| 16.2MB 291kB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.0-cp36-none-any.whl size=16173668 sha256=0d8a34c345060bff6944c201178661786fc7ad7903f5fb32d2fbf708b53eca71
  Stored in directory: /root/.cache/pip/wheels/fd/94/58/e8eb3fb96398ca699806937d09bcecb41bb9b877eab70ca741
Successfully built es-core-news-sm


In [3]:
#Ojo que esto cambia en Colab respecto al tutorial de SpaCy
import es_core_news_sm
nlp = es_core_news_sm.load()

text = (
    "De acuerdo con la revista Fortune, Apple fue la empresa "
    "más admirada en el mundo entre 2008 y 2012."
)

# Procesa el texto
doc = nlp(text)

# Imprime en pantalla el texto del documento
print(doc.text)

De acuerdo con la revista Fortune, Apple fue la empresa más admirada en el mundo entre 2008 y 2012.


###Ejercicio 5: Calcular el Part-of-speech y el dependency label de un texto

In [ ]:
# Procesa el texto
doc = nlp(text)

for token in doc:
    # Obtén el texto del token, el part-of-speech tag y el dependency label
    token_text = token.text
    token_pos = token.pos_
    token_dep = token.dep_
    # Esto es solo por formato
    print("{:<12}{:<10}{:<10}".format(token_text, token_pos, token_dep))

De          ADP       case      
acuerdo     NOUN      fixed     
con         ADP       fixed     
la          DET       det       
revista     NOUN      obl       
Fortune     PROPN     appos     
,           PUNCT     punct     
Apple       PROPN     nsubj     
fue         AUX       cop       
la          DET       det       
empresa     NOUN      ROOT      
más         ADV       advmod    
admirada    ADJ       amod      
en          ADP       case      
el          DET       det       
mundo       NOUN      obl       
entre       ADP       case      
2008        NOUN      obl       
y           CONJ      cc        
2012        NOUN      conj      
.           PUNCT     punct     


###Ejercicio 6: Extraer las entidades de un texto

In [ ]:
# Procesa el texto
doc = nlp(text)

# Itera sobre las entidades predichas
for ent in doc.ents:
    # Imprime en pantalla el texto de la entidad y su label
    print(ent.text, ent.label_)

Fortune ORG
Apple ORG


###Ejercicio 7: detectar entidades faltantes

In [ ]:
text = (
    "Los Olímpicos de Tokio 2020 son la inspiración para la nueva "
    "colección de zapatillas adidas zx."
)

# Procesa el texto
doc = nlp(text)


# Itera sobre las entidades
for ent in doc.ents:
    # Imprime en pantalla el texto de la entidad y su label
    print(ent.text, ent.label_)

# Obtén el span para "adidas zx"
adidas_zx = doc[14:16]

# Imprime en pantalla el texto del span
print("Entidad faltante:", adidas_zx.text)

Olímpicos de Tokio 2020 MISC
Entidad faltante: adidas zx


###Ejercicio 8: añadir entidades utilizando *patrones*

In [5]:
# Importa el Matcher
from spacy.matcher import Matcher


# Inicializa el matcher con el vocabulario compartido
matcher = Matcher(nlp.vocab)

# Crea un patrón que encuentre dos tokens: "adidas" y "zx"
pattern = [{"TEXT": "adidas"}, {"TEXT": "zx"}]

# Añade el patrón al matcher
matcher.add("ADIDAS_ZX_PATTERN", None, pattern)

# Usa al matcher sobre el doc
matches = matcher(doc)
print("Resultados:", [doc[start:end].text for match_id, start, end in matches])

Resultados: []


###Ejercicio 9: Crear patrones combinando letras y números y buscarlos en el texto

In [6]:
matcher = Matcher(nlp.vocab)

doc = nlp(
    "Después de hacer la actualización de iOS no notarás un rediseño "
    "radical del sistema: no se compara con los cambios estéticos que "
    "tuvimos con el iOS 7. La mayoría de las funcionalidades del iOS 11 "
    "siguen iguales en el iOS 10."
)

# Escribe un patrón para las versiones de iOS enteras
# ("iOS 7", "iOS 11", "iOS 10")
pattern = [{"TEXT": "iOS"}, {"IS_DIGIT": True}]

# Añade el patrón al matcher y usa el matcher sobre el documento
matcher.add("IOS_VERSION_PATTERN", None, pattern)
matches = matcher(doc)
print("Total matches found:", len(matches))

# Itera sobre los resultados e imprime el texto del span
for match_id, start, end in matches:
    print("Match found:", doc[start:end].text)

Total matches found: 3
Match found: iOS 7
Match found: iOS 11
Match found: iOS 10


Ojo con las diferentes versiones de los modelos preentrenados. Esto puede afectar al número de resultados obtenidos. En el tutorial de Spacy se obtienen algunas respuestas diferentes. 

In [7]:
matcher = Matcher(nlp.vocab)

doc = nlp(
    "descargué Fortnite en mi computadora, pero no puedo abrir el juego. "
    "Ayuda? Cuando estaba descargando Minecraft, conseguí la versión de Windows "
    "donde tiene una carpeta '.zip' y usé el programa por defecto para "
    "descomprimirlo…así que también tengo que descargar Winzip?"
)

# Escribe un patrón que encuentre una forma de "descargar" más un nombre propio
pattern = [{"LEMMA": "descargar"}, {"POS": "PROPN"}]

# Añade el patrón al matcher y usa el matcher sobre el documento
matcher.add("DOWNLOAD_THINGS_PATTERN", None, pattern)
matches = matcher(doc)
print("Total de resultados encontrados:", len(matches))

# Itera sobre los resultados e imprime el texto del span
for match_id, start, end in matches:
    print("Resultado encontrado:", doc[start:end].text)

Total de resultados encontrados: 3
Resultado encontrado: descargué Fortnite
Resultado encontrado: descargando Minecraft
Resultado encontrado: descargar Winzip


In [9]:
matcher = Matcher(nlp.vocab)

doc = nlp(
    "El gigante tecnológico IBM está ofreciendo lecciones virtuales "
    "sobre tecnologías avanzadas gratuitas en español."
)

# Escribe un patrón para un sustantivo más uno o dos adjetivos
pattern = [{"POS": "NOUN"}, {"POS": "ADJ"}, {"POS": "ADJ", "OP": "?"}]

# Añade el patrón al matcher y usa el matcher sobre el documento
matcher.add("NOUN_ADJ_PATTERN", None, pattern)
matches = matcher(doc)
print("Total de resultados encontrados:", len(matches))

# Itera sobre los resultados e imprime el texto del span
for match_id, start, end in matches:
    print("Resultado encontrado:", doc[start:end].text)

Total de resultados encontrados: 3
Resultado encontrado: gigante tecnológico
Resultado encontrado: lecciones virtuales
Resultado encontrado: tecnologías avanzadas
